In [1]:
import os

In [2]:
#os.environ['NCCL_DEBUG'] = 'INFO'
#os.environ['NCCL_DEBUG_SUBSYS'] = 'ALL'
#os.environ['TORCH_DISTRIBUTED_DEBUG'] = 'INFO'
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['NCCL_SOCKET_IFNAME'] = 'eno1'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_BLOCKING_WAIT'] = '1'

In [3]:
import sys
from os.path import join
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
import torch.multiprocessing as mp
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from astropy.io import fits
import pyxis.torch as pxt
from test_script import run_ddp

from networks import *
from train import *
from dataset import FiberDataset
import config

data_dir = '/xdisk/timeifler/wxs0703/kl_nn/train_data_massive/train_database'
test_dir = '/xdisk/timeifler/wxs0703/kl_nn/test_data/test_database'
samp_dir = '/xdisk/timeifler/wxs0703/kl_nn/samples/samples_massive.csv'
fig_dir = '/xdisk/timeifler/wxs0703/kl_nn/figures/'
model_dir = '/xdisk/timeifler/wxs0703/kl_nn/model/'

In [4]:
world_size = torch.cuda.device_count()
save_every = 1
nepochs = config.train['epoch_number']
batch_size = config.train['batch_size']
nfeatures = config.train['feature_number']

In [5]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
torch.cuda.set_device(0)
init_process_group(backend='nccl', rank=0, world_size=1)

[W410 11:17:56.440182833 Utils.hpp:164] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())


In [6]:
train_ds = pxt.TorchDataset(data_dir)
valid_ds = pxt.TorchDataset(test_dir)

In [7]:
train_dl = DataLoader(
        train_ds,
        batch_size=batch_size,
        pin_memory=True,
        sampler=DistributedSampler(train_ds),
    )
valid_dl = DataLoader(
        valid_ds,
        batch_size=batch_size,
        pin_memory=True,
        sampler=DistributedSampler(valid_ds),
    )

In [8]:
model = ForkCNN(batch_size)
model.to(0)
model = DDP(model, device_ids=[0])

[rank0]:[W410 11:17:57.047390329 Utils.hpp:110] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())


In [9]:
optimizer = optim.SGD(model.parameters(), 
                      lr=config.train['initial_learning_rate'],
                      momentum=config.train['momentum'])

In [10]:
trainer = Trainer(model, nfeatures, train_dl, valid_dl, optimizer, 0, save_every, batch_size)

In [11]:
trainer.train(50)

KeyboardInterrupt: 

In [6]:
destroy_process_group()

AssertionError: 

In [7]:
mp.spawn(train_nn, args=(world_size, DeconvNN, DeconvTrainer), nprocs=world_size)

INFO:Setup:Initializing
[W414 13:02:44.849839138 Utils.hpp:164] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())
INFO:Setup:[rank: 0] Successfully set up device
[rank0]:[W414 13:02:45.284929390 Utils.hpp:110] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())
INFO:Setup:[rank: 0] Successfully loaded training objects
INFO:Setup:[rank: 0] Successfully prepared dataloader
INFO:Setup:[rank: 0] Successfully initialized Trainer
INFO:Trainer:Batch 0 complete
INFO:Trainer:Batch 100 complete
INFO:Trainer:Batch 200 complete
INFO:Trainer:Batch 300 complete
INFO:Trainer:Batch 400 complete
INFO:Trainer:Batch 500 complete
INFO:Trainer:Batch 600 complete


KeyboardInterrupt: 